In [1]:
from __future__ import print_function
from net.models import SAVA_test as SAVA

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from PIL import Image
import matplotlib.pyplot as plt
from matplotlib import cm

import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

import argparse
import os
from tqdm import tqdm
import time

from net.network import AttentionNet, Encoder, SelfAttention, Transform, vgg, decoder
from net.utils import adaptive_instance_normalization

%load_ext autoreload
%autoreload 2

ImportError: cannot import name 'Transform' from 'net.network' (/public/dehezhang2/github/SAVA_Official_Implementation/net/network.py)

In [ ]:
parser = argparse.ArgumentParser()
# Basic options
# parser.add_argument('--content_image', type=str, default='./test_result/recon/lenna_recon_adain_content_wct.png',
#                     help='Directory path to a batch of content images')
parser.add_argument('--content_image', type=str, default='./datasets/test/content/lenna.jpg',
                    help='Directory path to a batch of content images')
parser.add_argument('--style_image', type=str, default='./datasets/test/void_style/xile.jpg',
                    help='Directory path to a batch of style images')

parser.add_argument('--attn_model', type=str, default='models/content_wct/',
                    help='Directory path to a batch of style images')
parser.add_argument('--decoder_model', type=str, default='models/SANET/',
                    help='Directory path to a batch of style images')
parser.add_argument('--transform_model', type=str, default='models/SANET/',
                    help='Directory path to a batch of style images')
parser.add_argument('--vgg_model', type=str, default='models/SANET/',
                    help='Directory path to a batch of style images')
# parser.add_argument('--save_dir', type=str, default='test_results/',
#                     help='Directory path to a batch of style images')
args = parser.parse_args('')

content_name = args.content_image.split('/')[-1].split('.')[0]
style_name = args.style_image.split('/')[-1].split('.')[0]
attn_model_dir = args.attn_model.split('/')[-2]
decoder_model_dir = args.decoder_model.split('/')[-2]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

_R_MEAN = 123.68
_G_MEAN = 116.78
_B_MEAN = 103.94

In [ ]:
# loading test data

def test_transform(size = 512):
    transform = transforms.Compose([
        transforms.Resize(size=(size, size)),
        transforms.CenterCrop(size),
        transforms.ToTensor(),
        transforms.Normalize((_R_MEAN/255.0, _G_MEAN/255.0, _B_MEAN/255.0), (0.5, 0.5, 0.5))
    ])
    return transform

def test_transform_inv():
    transform = transforms.Compose([
        transforms.Normalize((0, 0, 0), (2, 2, 2)),
        transforms.Normalize((-_R_MEAN/255.0, -_G_MEAN/255.0, -_B_MEAN/255.0), (1, 1, 1))
    ])
    return transform
# scale = 1.40625
content_tf = test_transform(int(512))
style_tf = test_transform(int(512))
content_tf_inv = test_transform_inv()
content = content_tf(Image.open(args.content_image).convert('RGB'))
print(content.shape)

style = style_tf(Image.open(args.style_image).convert('RGB'))
print(style.shape)

import matplotlib.pyplot as plt
import numpy as np

# functions to show an image
def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

imshow(torchvision.utils.make_grid(content_tf_inv(content)))

In [ ]:
imshow(torchvision.utils.make_grid(content_tf_inv(style)))

In [ ]:
decoder = decoder
decoder.load_state_dict(torch.load(args.decoder_model + 'decoder_iter_500000.pth'))
attn = SelfAttention()
attn.load_state_dict(torch.load(args.attn_model + 'attention_kernel_iter_80000.pth'))
transform = Transform(in_planes=512)
transform.load_state_dict(torch.load(args.transform_model + 'transformer_iter_500000.pth'))
vgg = vgg
vgg.load_state_dict(torch.load(args.vgg_model + 'vgg_normalised.pth'))
encoder = Encoder(encoder = vgg)

state_dict = decoder.state_dict()
for key in state_dict.keys():
    state_dict[key] = state_dict[key].to(device)
state_dict = attn.state_dict()
for key in state_dict.keys():
    state_dict[key] = state_dict[key].to(device)
state_dict = transform.state_dict()
for key in state_dict.keys():
    state_dict[key] = state_dict[key].to(device)
state_dict = vgg.state_dict()
for key in state_dict.keys():
    state_dict[key] = state_dict[key].to(device)

transform.cuda()

model = AttentionNet(attn=attn, encoder = encoder, decoder=decoder)
model.cuda()

In [ ]:
content = torch.stack([content], dim = 0)
content = content.to(device)
style = torch.stack([style], dim = 0)
style = style.to(device)

In [ ]:
sava = SAVA(model, transform)

In [ ]:
output, content_attn_map, style_attn_map= sava.transfer(content, style)
output = adaptive_instance_normalization(output, style)

## Show and Save Results

In [ ]:
def imshow_recon(img, save_dir):
    npimg = img.detach().numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
#     plt.savefig(save_dir)
#     torchvision.utils.save_image(img, save_dir)
    
def show_output(output, save_dir):
    print("Saving to " + save_dir)
    content_result = content_tf_inv(output.squeeze())
    imshow_recon(torchvision.utils.make_grid(content_result.cpu()), save_dir)

In [ ]:
show_output(output, 'test_result/output/' + content_name + '_with_' + style_name + '_' + attn_model_dir + '_' + decoder_model_dir + '.png')

In [ ]:
def save_attn_map(attn_map, save_dir):
    channel_num = attn_map.size()[1]
    mean_sal = torch.mean(attn_map, 1, False)
    mean_sal_np = mean_sal.cpu().detach().numpy()
    mean_sal_np = mean_sal_np - np.min(mean_sal_np)
    mean_sal_np = mean_sal_np * 1.0 / np.max(mean_sal_np)
    # print(mean_sal.size())
    plt.imshow(mean_sal_np[0], cmap=cm.get_cmap('rainbow', 1000))
    plt.colorbar()
    plt.axis('off')
#     plt.savefig(save_dir)

In [ ]:
save_attn_map(content_attn_map, 'test_result/attention/' + content_name + '_attn_' + attn_model_dir + '_' + decoder_model_dir + '.png')
print(content_attn_map.shape)

In [ ]:
save_attn_map(style_attn_map, 'test_result/attention/' + style_name + '_attn_' + attn_model_dir + '_' + decoder_model_dir + '.png')